## The Culture of International Relations - Text Analysis
### <span style='color: green'>SETUP </span> Prepare and Setup Notebook <span style='float: right; color: red'>MANDATORY</span>

In [1]:
# Setup
%load_ext autoreload
%autoreload 2

import sys
import ipywidgets as widgets

sys.path = list(set(['..', '../3_text_analysis']) - set(sys.path)) + sys.path

import matplotlib.pyplot as plt
import common.utility as utility
import common.treaty_utility as treaty_utility
import common.treaty_state as treaty_repository
import treaty_corpus
import textacy_corpus_utility as textacy_utility
from beakerx.object import beakerx
from beakerx import *
from IPython.display import display, set_matplotlib_formats

logger = utility.getLogger('corpus_text_analysis')

treaty_repository.load_wti_index_with_gui(data_folder=config.DATA_FOLDER)

%matplotlib inline



2020-03-20 09:27:23,291 : INFO : treaty_state.py.load_wti_index() : WTI index loaded!


In [2]:
import zipfile
with zipfile.ZipFile('../data/tCoIR_en_45-72_renamed.txt.zip', 'w', compression=zipfile.ZIP_DEFLATED) as new_zipfile:
    with zipfile.ZipFile('../data/tCoIR_en_45-72.txt.zip') as old_zipfile:
        for filename in old_zipfile.namelist():
            treaty_id = filename.split('_')[0]
            signed_year = treaty_repository.current_wti_index().treaties.loc[treaty_id]['signed_year']
            new_filename = '{}_{}'.format(signed_year, filename)
            with old_zipfile.open(filename) as textfile:
                content = textfile.read()
            new_zipfile.writestr(new_filename, content)